<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/5_Word2Vec_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec

- Gensim 패키지를 사용한 Word2Vec
- 참고URL: https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

1) 학습에 필요한 코퍼스 수집

In [1]:
!pip3 install wikipedia-api

In [2]:
!pip3 install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.5 MB/s eta 0:00:00


In [7]:
# 설치된 라이브리 호출
import wikipediaapi
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt


# 페이지 요약 정보를 수집
#wiki_txt=wikipediaapi.Wikipedia('ko')

wiki_txt = wikipediaapi.Wikipedia(user_agent='MyProjectName (merlin@example.com)', language='ko')
search_nm = '경영'

# 해당 페이지가 존재하는지 확인
page_py = wiki_txt.page(search_nm)
print("Page - Exists: %s" % page_py.exists())

# 페이지의 내용 전체를 수집
wiki = wikipediaapi.Wikipedia( user_agent='MyProjectName (merlin@example.com)',language='ko', extract_format=wikipediaapi.ExtractFormat.WIKI)
p_wiki = wiki.page(search_nm)
print(p_wiki.text)

Page - Exists: True
경영(經營, 영어: management)은 '조직에 관한 의사결정을 행하는 행동'을 말한다.경영이라는 말은 대한민국의 학계나 실업계에서 그 개념이 극히 모호하게 또 추상적으로 사용되고 있다. 경영한다는 것은 어느 경제단위를 그 설립목적에 부합하도록 의식적으로 계획·유도하고 지휘하는 것을 말한다. 예를 들면 학교·병원·고아원 등과 같은 비영리적 경제단위나 영리적 경제단위인 주식회사를 경영한다는 것은 이들 경영체의 설립목적에 부응하도록 계획하고 낭비없이 밀고간다는 것을 말한다. 또한 기업이라는 말도 역시 구분되어 있지 않은데, 기업이란 위험성을 인식하고도 그 위험성을 극복, 더욱 큰 이윤을 추구하고자 하는 그 경제단위의 설립목적을 달성하려고 시도하는 것을 말한다. 이렇게 볼 때 모든 경제단위(비영리적 경제단위와 영리를 목적으로 하는 경제단위)는 경영체이며, 특히 영리를 목적으로 경영하는 경제단위를 기업체라고 부른다. 물론 기업체를 경영한다는 것은 기업체의 목적달성이 성취되도록 시도한다는 것을 뜻한다.

의미
경영이란 개념의 변화
경영이란 말은 오래전부터 있었으나 여기서 사용하는 의미는 새로운 개념이다.

규모(規模)를 정하고 기초를 세워 일을 해나감.
계획을 세워 사업을 해나감.
〔경〕계속적 사업을 경제적으로 해나가기 위한 작업상의 조직.

여기서 첫 번째 뜻은 고전적인 용어로, 앞으로 말할 경영과는 관계가 없다. 두 번째 뜻은 경영에 대한 동태적(動態的)·형태적(形態的)인 이해이다. 이 둘은 구체적으로는 관계가 있지만 그 지향하는 바는 서로 다르다.
경영의 본래 의미는 조직을 관리하고 운영하는 것이다. 이를 조직의 목적과 연관시켜 '조직의 목적을 달성하기 위한 일체의 과정'이라고도 할 수 있다. 앞의 쓰인 조직은 가계, 기업, 기관 또는 다른 어떤 조직도 될 수도 있다. 하지만, 현대사회에 접어들면서 기업의 역할과 비중이 점차 커지게 되어 오늘날에는 주로 기업 경영(corporate management)을 의미하게 되었다.



2) 데이터 전처리
- 문장 추출, 불용어 제거

In [8]:
import nltk #자연어 처리 패키지
from nltk.corpus import stopwords #NLTK에서 제공되는 불용어 사전
import string # Python  문자열 처리 클래스
import pandas as pd # 데이터분석 라이브러리
import re
from nltk.tokenize import word_tokenize, sent_tokenize

stopwords_kor = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def process_text(text):
    #1 문자 구분자 제거
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

    #2 영어 불용어 제거
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

    #3 한글 불용어 제거
    clean_words = [word for word in clean_words if not word in stopwords_kor] # 불용어 제거

    #3 리스트 반환
    return clean_words

In [9]:
# 전체 문장 전처리 및 문단 기준 분리
from textblob import TextBlob

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from konlpy.tag import Okt
okt = Okt()

tokenized_data = []
text = p_wiki.text

blob = TextBlob(text) # TextBolob 객체 생성

#토큰화(문장기준) 처리
for sentence in blob.sentences:
    temp = " ".join(process_text(sentence))

    temp = okt.nouns(temp)
    temp = [word for word in temp if len(word) > 1]
    print(temp)

    tokenized_data.append(temp)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['경영', '영어', '조직', '의사결정', '행동', '경영', '대한민국', '학계', '실업계', '개념', '극히', '추상', '사용']
['단위', '설립', '목적', '부합', '의식', '계획', '유도', '지휘']
['학교', '병원', '고아원', '비영리', '단위', '영리', '단위', '주식회사', '설립', '목적', '부응', '계획', '낭비', '밀고', '간다']
['또한', '기업', '역시', '구분', '기업', '위험성', '인식', '위험성', '극복', '더욱', '이윤', '추구', '단위', '설립', '목적', '달성', '시도']
['모든', '단위', '비영리', '단위', '영리', '목적', '단위', '영리', '목적', '경영', '단위', '기업체']
['기업체', '기업체', '목적', '달성', '성취', '시도']
['의미', '경영', '개념', '변화', '경영', '부터', '여기', '사용', '의미', '개념']
['규모', '기초', '해나']
['계획', '사업', '해나']
['계속', '사업', '경제', '해나', '가기', '작업', '상의', '조직']
['여기', '전적', '용어', '경영', '관계']
['경영', '대한', '동태', '형태', '이해']
['구체', '관계', '지향', '서로']
['경영', '본래', '의미', '조직', '관리', '운영']
['조직', '목적', '연관', '조직', '목적', '달성', '일체', '과정']
['조직', '가계', '기업', '기관', '다른', '조직', '수도']
['사회', '기업', '역할', '비중', '점차', '오늘날', '주로', '기업', '경영', '의미']
['영어', '경영', '경영', '단어', '영어', '거나', '불리', '기도']
['어원', '생존', '발전', '위해', '매우', '활동', '의미']
['조합', '생존', '발전', '목적', '달성', '최고'

3) Word2Vec 모델생성 및 주요파라미터
- min_count : 최소단어의 빈도
- window: 현재 단어와 예측 단어 사이의 최대 거리
- size: 피처 벡터의 차원
- negative: 0 이상인 경우 네거티브 샘플링이 사용됨
- workers: 작업자의 스레드 갯수

In [35]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size= 200, window = 5, negative = 2, min_count = 1, workers = 4, sg = 1)

# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(1363, 200)

4) 모델 사용

In [12]:
print(model.wv.most_similar("경영"))

[('증가', 0.3870247006416321), ('다른', 0.3821452856063843), ('종업원', 0.3293178677558899), ('중심', 0.31392914056777954), ('프로세스', 0.31172654032707214), ('조직', 0.3030332326889038), ('공업', 0.2988959848880768), ('업무', 0.2868540585041046), ('산업', 0.28462058305740356), ('기계', 0.2841964066028595)]


## FastText

In [21]:
from gensim.models import FastText
ft_model = FastText(tokenized_data, vector_size= 200, window=5, negative = 2,min_count=1, workers=4, sg=1)

In [22]:
print(ft_model.wv.most_similar("경영"))

[('경영학', 0.49138471484184265), ('노사', 0.3756369650363922), ('무엇', 0.3251892924308777), ('경영인', 0.31775832176208496), ('노동조합', 0.29332494735717773), ('책임', 0.29014042019844055), ('기술', 0.2745293378829956), ('형태', 0.2705365717411041), ('대한', 0.26135745644569397), ('기업', 0.2565551698207855)]
